In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import category_encoders as ce

In [2]:
path="../Data/"
os.listdir(path)

['Encoded', 'Fixated.csv', 'Mentah.xlsx', 'Weusethis.csv']

Label Encode!

In [3]:
#Get df
df=pd.read_csv(path+"Weusethis.csv")

#Change tolower
df["KELURAHAN"].unique()
df["KELURAHAN"]=df['KELURAHAN'].map(str.lower)

encoder={}
#Change target to binary 
targetname=['TIDAK BANJIR', 'BANJIR']
for i in range(len(targetname)):
    df["Kejadian"][df["Kejadian"]==targetname[i]]=i 
df["Kejadian"]=df["Kejadian"].astype("int64")
encoder["Kejadian"]=dict(zip([str(i) for i in range(2)], targetname))

#Turn Rain column with ["Yes" / "No"] value to binary
temp=["NO", "YES"]
for i in df.columns:
    checker=0
    for j in range(len(temp)):
        if temp[j] in df[i].unique() : df[i][df[i]==temp[j]]=j; checker=1
    if checker==1 : encoder[i]=dict(zip([str(x) for x in range(2)], temp))

#Turn TEPS_OF_DAY to ordinal, sorted from no rain
temp=["TIDAK HUJAN", "HUJAN RINGAN", "HUJAN SEDANG", "HUJAN LEBAT"]
for i in range(len(temp)):
    df=df[df!=temp[i]].fillna(i)
encoder["TEPS_OF_DAY"]=dict(zip([str(x) for x in range(len(temp))], temp))

C:\Users\Rhey\AppData\Local\Temp\ipykernel_18364\271395425.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Kejadian"][df["Kejadian"]==targetname[i]]=i
C:\Users\Rhey\AppData\Local\Temp\ipykernel_18364\271395425.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if temp[j] in df[i].unique() : df[i][df[i]==temp[j]]=j; checker=1
C:\Users\Rhey\AppData\Local\Temp\ipykernel_18364\271395425.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if temp[j] in df[i].unique() : df[i][df[i]==temp[j]]=j; checker=1


## Unhandled Data

In [4]:
#change "keluarahan" as one hots
onehotencoder=ce.OneHotEncoder(cols=["KELURAHAN"], use_cat_names=True)
unhandleddata=onehotencoder.fit_transform(df)

In [5]:
#unhandleddata save
unhandleddata.to_csv(path+"Encoded/undhandled.csv")

## Handle Imbalanced
### Resampling

In [6]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = df[df['Kejadian']==0] 
df_minority = df[df['Kejadian']==1] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= len(df_majority), # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [7]:
#Check tot distribution by each "kelurahan"
thelist=[]
for i in df_upsampled["KELURAHAN"].unique():
    temp=df_upsampled.iloc[:,:2].values
    temp=temp[temp[:,1]==i]
    len(temp[temp[:,0]==1])
    thelist.append([i, len(temp[temp[:,0]==0]), len(temp[temp[:,0]==1]), len(temp[temp[:,0]==0])+len(temp[temp[:,0]==1])])
pd.DataFrame(thelist, columns=["kelurahan", "n tidak banjir", "n banjir", "Total"])

,kelurahan,n tidak banjir,n banjir,Total
0,halim perdana kusuma,10,5,15
1,penggilingan,10,13,23
2,duren sawit,20,33,53
3,batu ampar,20,19,39
4,jatinegara,20,32,52
5,cipinang melayu,30,21,51
6,pisangan timur,10,12,22
7,bidara cina,30,48,78
8,kampung melayu,30,83,113
9,rawa bunga,20,7,27


In [8]:
df_upsampled[df_upsampled["KELURAHAN"]=="halim perdana kusuma"]

,Kejadian,KELURAHAN,RAIN_D-1,RAINFALL_D-1,RAIN_LAST3D,ARAINFALL_LAST3D,RAIN_LAST7D,RAINFALL_LAST7D,TEPS_OF_DAY,CIPINANG HULU,SUNTER HULU,PULO GADUNG
102,1,halim perdana kusuma,1,39.8,1,26.3,1,101.8,2,115,200,735
102,1,halim perdana kusuma,1,39.8,1,26.3,1,101.8,2,115,200,735
102,1,halim perdana kusuma,1,39.8,1,26.3,1,101.8,2,115,200,735
102,1,halim perdana kusuma,1,39.8,1,26.3,1,101.8,2,115,200,735
102,1,halim perdana kusuma,1,39.8,1,26.3,1,101.8,2,115,200,735
575,0,halim perdana kusuma,0,0.0,0,0.0,0,33.0,0,95,80,330
576,0,halim perdana kusuma,0,0.0,0,0.0,0,2.5,0,80,70,330
577,0,halim perdana kusuma,0,0.0,0,0.0,0,0.0,0,75,50,330
578,0,halim perdana kusuma,1,14.0,0,12.0,0,13.0,1,110,70,370
579,0,halim perdana kusuma,0,0.0,0,0.0,0,23.5,0,75,50,400


In [9]:
df_upsampled.to_csv(path+"Encoded/upsampled.csv")

 That's just a shit way to get more data, i think :Sob:. Basically just multiplcating the same data.

### SMOTE

In [57]:
from imblearn.over_sampling import SMOTE

#All cols Needs to be changed to  numeric :/
temp=df.copy()
lala=ce.OrdinalEncoder(cols=["KELURAHAN"]).fit(temp)
temp=lala.transform(temp)

sm = SMOTE(sampling_strategy='minority', random_state=42)
# Fit the model to generate the data.
oversampled_X, oversampled_Y = sm.fit_resample(temp.drop('Kejadian', axis=1), temp['Kejadian'])
oversampled = pd.concat([pd.DataFrame(oversampled_Y), pd.DataFrame(oversampled_X)], axis=1)

#turn shit back
temp=dict(zip(list(lala.category_mapping[0]["mapping"]),lala.category_mapping[0]["mapping"].index))
for i in temp.keys():
    oversampled["KELURAHAN"][oversampled["KELURAHAN"]==i]=temp[i]

C:\Users\Rhey\AppData\Local\Temp\ipykernel_18364\373192988.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oversampled["KELURAHAN"][oversampled["KELURAHAN"]==i]=temp[i]


In [60]:
#Check tot distribution by each "kelurahan"
thelist=[]
for i in oversampled["KELURAHAN"].unique():
    temp=oversampled.iloc[:,:2].values
    temp=temp[temp[:,1]==i]
    len(temp[temp[:,0]==1])
    thelist.append([i, len(temp[temp[:,0]==0]), len(temp[temp[:,0]==1]), len(temp[temp[:,0]==0])+len(temp[temp[:,0]==1])])
pd.DataFrame(thelist, columns=["kelurahan", "n tidak banjir", "n banjir", "Total"])

,kelurahan,n tidak banjir,n banjir,Total
0,balekambang,30,17,47
1,bambu apus,10,9,19
2,batu ampar,20,19,39
3,bidara cina,30,27,57
4,cakung timur,10,22,32
5,cawang,30,41,71
6,cililitan,30,31,61
7,cipayung,20,18,38
8,cipinang,10,18,28
9,cipinang cempedak,20,20,40


In [64]:
oversampled[oversampled["KELURAHAN"]=="halim perdana kusuma"]

,Kejadian,KELURAHAN,RAIN_D-1,RAINFALL_D-1,RAIN_LAST3D,ARAINFALL_LAST3D,RAIN_LAST7D,RAINFALL_LAST7D,TEPS_OF_DAY,CIPINANG HULU,SUNTER HULU,PULO GADUNG
102,1,halim perdana kusuma,1,39.800000,1,26.300000,1,101.800000,2,115,200,735
575,0,halim perdana kusuma,0,0.000000,0,0.000000,0,33.000000,0,95,80,330
576,0,halim perdana kusuma,0,0.000000,0,0.000000,0,2.500000,0,80,70,330
577,0,halim perdana kusuma,0,0.000000,0,0.000000,0,0.000000,0,75,50,330
578,0,halim perdana kusuma,1,14.000000,0,12.000000,0,13.000000,1,110,70,370
579,0,halim perdana kusuma,0,0.000000,0,0.000000,0,23.500000,0,75,50,400
580,0,halim perdana kusuma,0,0.000000,0,33.500000,0,80.000000,0,90,100,330
581,0,halim perdana kusuma,0,0.000000,0,0.000000,0,0.000000,0,80,55,330
582,0,halim perdana kusuma,1,4.000000,0,0.000000,0,68.500000,1,90,90,330
583,0,halim perdana kusuma,1,1.000000,0,0.000000,0,0.000000,1,85,50,330


Ye the same as resampling shit.

###